# Adiabatic SSM model

## 1. Distance weighting

This notebook takes multiple local SSM models which have been identified at pre-tensioned equilibria and interpolates them using distance weighting to obtain one adiabatic SSM model. The model performance is evaluated by its ability to predict open-loop trajectories.

In [ ]:
from os import listdir
from os.path import join
import pickle
import yaml
import numpy as np
np.set_printoptions(linewidth=100)

In [ ]:
%load_ext autoreload
%autoreload 2
import utils as utils
import plot_utils as plot

In [ ]:
# %matplotlib qt
import matplotlib.pyplot as plt

Import local SSM models

In [ ]:
# directory containing the SSM models
model_dir = "/media/jonas/Backup Plus/jonas_soft_robot_data/trunk_adiabatic"
# model_names = sorted(listdir(model_dir))
model_names = ["north","west", "origin", "east", "south"] # 
print(model_names)
N_models = len(model_names)

Load the pre-tensioned equilibrium of each model and plot adiabatic SSM map

In [ ]:
q_eq, u_eq = [], []
for model in model_names:
    with open(join(model_dir, model, "SSMmodel", "pre-tensioned_rest_q.pkl"), "rb") as f:
        q_eq.append(pickle.load(f))
    with open(join(model_dir, model, "decay", "info.yaml"), "r") as f:
        info = yaml.safe_load(f)
    u_eq.append(np.array(info['pre_tensioning']))
print(q_eq)
print(u_eq)
xy_rest = np.array([q_eq[i][:2] for i in range(N_models)])

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, dpi=120)

for i, model in enumerate(model_names):
    ax.plot(q_eq[i][0], q_eq[i][1], marker="D", markersize=8, ls="", label=model)
ax.axhline(0, ls='--', color='grey', zorder=1, alpha=.6, lw=0.5)
ax.axvline(0, ls='--', color='grey', zorder=1, alpha=.6, lw=0.5)
ax.set_xlabel(r"$x$ [mm]")
ax.set_ylabel(r"$y$ [mm]")
ax.set_aspect("equal")
ax.legend()
plt.show()

### Model weighting

In [ ]:
DECAY_EXP = 2

In [ ]:
import matplotlib as mpl

nx, ny = (100, 100)
grid_lims = (-50, 50)
x = np.linspace(grid_lims[0], grid_lims[1], nx)
y = np.linspace(grid_lims[0], grid_lims[1], ny)
xv, yv = np.meshgrid(x, y, sparse=False)

model_weights = None
colormaps = ['Blues', 'Oranges', 'Greens', 'Reds', 'Purples']

xy_rest = np.array([q_eq[i][:2] for i in range(len(model_names))])

if DECAY_EXP == np.inf:
    model_weights = np.zeros((len(model_names), nx, ny))
    active_model = np.argmin(np.linalg.norm(np.stack([xy_rest[i, :] - np.stack((xv, yv), axis=-1) for i in range(len(model_names))]), axis=-1), axis=0)
    for i in range(nx):
        for j in range(ny):
            model_weights[active_model[i, j], i, j] = 1
    
else:
    model_weights = 1 / np.linalg.norm(np.stack([xy_rest[i, :] - np.stack((xv, yv), axis=-1) for i in range(len(model_names))]), axis=-1) ** DECAY_EXP
    model_weights /= np.sum(model_weights, axis=0)

fig, axs = plt.subplots(1, len(model_names), figsize=(16, 3), dpi=150, sharey=True, constrained_layout=True)
im = None
for i, model in enumerate(model_names):
    ax = axs[i]
    im = ax.contourf(x, y, model_weights[i, :, :],
                    vmin=0, vmax=1, levels=10) # cmap=mpl.colormaps[colormaps[i]])
    ax.set_title(model)
    ax.set_xlabel(r"$x$ [mm]")
    ax.set_aspect("equal")
axs[0].set_ylabel(r"$y$ [mm]")
fig.colorbar(im, ax=axs.ravel().tolist())
plt.show()

In [ ]:
def interpolate_coeffs(xy, coeffs: list):
    # print(xy.ndim)
    if xy.ndim > 1:
        weights = np.stack([1 / np.linalg.norm(xy_rest - xy[:, i], axis=1) ** DECAY_EXP for i in range(xy.shape[1])]).T
        weights /= np.sum(weights, axis=0)
        return np.tensordot(weights.T, np.stack(coeffs), axes=1)
    else:
        if DECAY_EXP == np.inf:
            weights = np.zeros(len(model_names))
            active_model = np.argmin(np.linalg.norm(xy_rest - xy, axis=1))
            weights[active_model] = 1
        else:
            weights = 1 / np.linalg.norm(xy_rest - xy, axis=1) ** DECAY_EXP
            weights /= np.sum(weights)
        return np.sum([coeffs[i] * weights[i] for i in range(len(coeffs))], axis=0), weights
    # return coeffs[2] # use only model from origin

# coeffs = [np.random.random((7, 12)) for i in range(5)]
# print(interpolate_coeffs(np.zeros((2, 4)), coeffs).shape)

Define mappings of adiabatic SSM

In [ ]:
models = []
for model_name in model_names:
    with open(join(model_dir, model_name, "SSMmodel", "SSM_model.pkl"), "rb") as f:
        model = pickle.load(f)
        models.append(model)
Vde = [model['model']['V'] for model in models]
r_coeff = [model['model']['r_coeff'] for model in models]
w_coeff = [model['model']['w_coeff'] for model in models]
B = [model['model']['B'] for model in models]
ROMOrder = [model['params']['ROM_order'] for model in models]
SSMOrder = [model['params']['SSM_order'] for model in models]

In [ ]:
print(w_coeff[0].shape)

#### Prediction of open-loop trajectories

In [ ]:
observables = "delay-embedding" # "pos-vel" # 
N_DELAY = 4 # only relevant if observables is "delay-embedding"
TIP_NODE = 51
N_NODES = 709
INPUT_DIM = 8
DT = 0.01

rDOF = 3
oDOF = 3
SSMDim = 6

robot_dir = "../../../soft-robot-control/examples/trunk"
data_dir = "/media/jonas/Backup Plus/jonas_soft_robot_data/trunk_adiabatic/origin"
rest_file = join(robot_dir, 'rest_qv.pkl')

Compute observables (delay embedding)

In [ ]:
N_DELAY = 4
assemble_observables = lambda oData: utils.delayEmbedding(oData, up_to_delay=N_DELAY)

Predict open-loop trajectories using the interpolated adiabatic SSM model

In [ ]:
test_results = {}
test_trajectory_folders = ['open-loop_circle']
test_trajectories = []
for test_traj in test_trajectory_folders:
    traj_dir = join(robot_dir, "dataCollection", test_traj)
    (t, z), u = utils.import_pos_data(data_dir=traj_dir,
                                      rest_file=rest_file,
                                      output_node=TIP_NODE, return_inputs=True, traj_index=0)
    # z = (z.T - q_eq).T
    # u = (u.T - u_eq).T
    y = assemble_observables(z)
    print("y.shape:", y.shape)
    print("u.shape:", u.shape)

    test_trajectories.append({
            'name': test_traj,
            't': t,
            'z': z,
            'u': u,
            'y': y
        })

In [ ]:
def advect_adiabaticRD_with_inputs(t, y0, u, y, know_target=True):
    dt = 0.01
    N = len(t)-1
    x = np.zeros((6, N+1))
    x[:, 0] = [5, 5, 5, 0, 0, 0]
    y_pred = np.zeros((15, N+1))
    if not know_target:
        y = y_pred
    y_bar = np.zeros((15, N+1))
    u_bar = np.zeros((8, N+1))
    xdot = np.zeros((6, N+1))
    y_pred[:, 0] = y0
    weights = np.zeros((len(model_names), N+1))
    for i in range(N):
        try:
            weights[:, i] = interpolate_coeffs(y[:2, i], [1])[1]
            # y_bar[:, i] = np.tile(interpolate_coeffs(y[:2, i], q_eq)[0], 5)
            # u_bar[:, i] = interpolate_coeffs(y[:2, i], u_eq)[0]
            # print(y[:2, i])
            # x[:, i] = interpolate_coeffs(y[:2, i], Vde)[0].T @ (y_pred[:, i].T - y_bar[:, i])
            # print("B @ u.shape:", (interpolate_coeffs(y[:2, i], B) @ u[:, i]).shape)
            xdot[:, i] = -0.5 * np.eye(6) @ x[:, i] # (interpolate_coeffs(y[:2, i], r_coeff)[0] @ utils.phi(x[:, i].reshape(-1, 1), ROMOrder[0])).flatten() # + interpolate_coeffs(y[:2, i], B)[0] @ (u[:, i] - u_bar[:, i])
            x[:, i+1] = x[:, i] + dt * xdot[:, i]
            y_pred[:, i+1] = (interpolate_coeffs(y[:2, i], w_coeff)[0] @ utils.phi(x[:, i+1].reshape(-1, 1), SSMOrder[0])).T + y[:, i] # y_bar[:, i]
        except Exception as e:
            print(e)
            break
    return x, y_pred, xdot, y_bar, u_bar, weights

In [ ]:
traj = test_trajectories[0]
print(traj['name'])
x, y, xdot, y_bar, u_bar, weights = advect_adiabaticRD_with_inputs(traj['t'], traj['y'][:, 0], traj['u'], traj['y'], know_target=True)

In [ ]:
plt.close('all')
# Trajectory xyz vs. t (predicted vs. actual)
axs = plot.traj_xyz_txyz(traj['t'],
                            y[0, :], y[1, :], y[2, :],
                            show=False)
axs = plot.traj_xyz_txyz(traj['t'],
                        traj['z'][0, :], traj['z'][1, :], traj['z'][2, :],
                        color="tab:orange", axs=axs, show=False)
axs[-1].legend(["Predicted trajectory", "Actual trajectory"])
# for ax in axs:
#     ax.set_ylim(-35, 35)
plt.suptitle(r"Trajectory (predicted vs. actual)")
plt.show()

# Trajectory 2D (predicted vs. actual)
ax = plot.traj_2D_xy(y[0, :], y[1, :],
                      show=False)
ax = plot.traj_2D_xy(traj['z'][0, :], traj['z'][1, :],
                        color="tab:orange", ax=ax, show=False)
ax.legend(["Predicted trajectory", "Actual trajectory"])
# ax.set_xlim(-35, 35)
# ax.set_ylim(-35, 35)
plt.suptitle(r"Trajectory (predicted vs. actual)")
plt.show()

# weights
axs = plot.adiabatic_model_weights(traj['t'], weights, model_names)

# y_bar
axs = plot.traj_xyz_txyz(traj['t'],
                            y_bar[0, :], y_bar[1, :], y_bar[2, :],
                            xyz_names=[r"$\bar{y_1}$", r"$\bar{y_2}$", r"$\bar{y_3}$"],
                            show=False)
plt.suptitle(r"$\bar{y}}$")
plt.show()


### 2. Scattered interpolation

In [ ]:
# directory containing the SSM models
model_dir = "/media/jonas/Backup Plus/jonas_soft_robot_data/trunk_adiabatic"
# model_names = sorted(listdir(model_dir))
model_names = ["northwest", "north", "northeast", "west", "origin", "east", "southwest", "south", "southeast"] # ["north","west", "origin", "east", "south"] # 
print(model_names)
N_models = len(model_names)

Load the pre-tensioned equilibrium of each model and plot adiabatic SSM map

In [ ]:
q_eq, u_eq = [], []
for model in model_names:
    with open(join(model_dir, model, "SSMmodel", "pre-tensioned_rest_q.pkl"), "rb") as f:
        q_eq.append(pickle.load(f))
    with open(join(model_dir, model, "decay", "info.yaml"), "r") as f:
        info = yaml.safe_load(f)
    u_eq.append(np.array(info['pre_tensioning']))
print(q_eq)
print(u_eq)
xy_rest = np.array([q_eq[i][:2] for i in range(N_models)])

In [ ]:
models = []
for model_name in model_names:
    with open(join(model_dir, model_name, "SSMmodel", "SSM_model.pkl"), "rb") as f:
        model = pickle.load(f)
        models.append(model)
Vde = [model['model']['V'] for model in models]
r_coeff = [model['model']['r_coeff'] for model in models]
w_coeff = [model['model']['w_coeff'] for model in models]
B = [model['model']['B'] for model in models]
ROMOrder = [model['params']['ROM_order'] for model in models]
SSMOrder = [model['params']['SSM_order'] for model in models]

Compute interpolations: Delaunay triangulation, Barycentric linear interpolation

In [ ]:
from scipy.interpolate import NearestNDInterpolator
from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import CloughTocher2DInterpolator
from scipy.spatial import Delaunay

interpolator = LinearNDInterpolator

# xy_rest[0][0] -= 2

# compute delauny triangulation attached to the pre-tensioned equilibria
tri = Delaunay(xy_rest)
# create interpolants for the different coefficient matrices
interpolation = {}
for name, coeffs in [('W', w_coeff), ('V', Vde), ('R', r_coeff), ('B', B), ('u_bar', u_eq), ('q_bar', q_eq)]:
    interpolation[name] = interpolator(tri, coeffs)

Visualize the interpolation scheme

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=150)
ax.triplot(xy_rest[:,0], xy_rest[:,1], tri.simplices.copy(), color="black", ls="-", lw=0.8)
for i in range(N_models):
    ax.plot(xy_rest[i,0], xy_rest[i,1], marker="D", markersize=8, ls="", color="tab:blue")
ax.set_aspect("equal")
ax.set_xlabel(r"$x$ [mm]")
ax.set_ylabel(r"$y$ [mm]")
# fig.suptitle("Delaunay triangulation")
plt.show()

In [ ]:
nx, ny = (100, 100)
grid_lims = (-40, 40)
x = np.linspace(grid_lims[0], grid_lims[1], nx)
y = np.linspace(grid_lims[0], grid_lims[1], ny)
# xv, yv = np.meshgrid(x, y, sparse=False)

model_weights = np.zeros((N_models, nx, ny))
for i in range(nx):
    for j in range(ny):
        p = [x[i], y[j]]
        simplex = tri.find_simplex(p)
        if simplex == -1:
            # extrapolation would be necessary
            model_weights[:, j, i] = np.nan
        else:
            b = tri.transform[simplex,:2] @ (p - tri.transform[simplex,2])
            c = np.r_[b, 1 - b.sum()]
            point_idx = tri.simplices[simplex]
            model_weights[point_idx, j, i] = c

In [ ]:
model_weights.shape

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(10, 10), dpi=100, sharey=True, sharex=True, constrained_layout=True)
im = None
axs = axs.ravel()
for i, model in enumerate(model_names):
    ax = axs[i]
    im = ax.contourf(x, y, model_weights[i, :, :],
                    vmin=0, vmax=1, levels=100) # cmap=mpl.colormaps[colormaps[i]])
    # ax.set_title(model)
    ax.set_xlabel(r"$x$ [mm]")
    ax.set_aspect("equal")
for ax in axs[[0, 3, 6]]:
    ax.set_ylabel(r"$y$ [mm]")
fig.colorbar(im, ax=axs.ravel().tolist(), fraction=0.03, aspect=35)
plt.show()

In [ ]:
observables = "delay-embedding" # "pos-vel" # 
N_DELAY = 4 # only relevant if observables is "delay-embedding"
TIP_NODE = 51
N_NODES = 709
INPUT_DIM = 8
DT = 0.01

rDOF = 3
oDOF = 3
SSMDim = 6

robot_dir = "../../../soft-robot-control/examples/trunk"
data_dir = "/media/jonas/Backup Plus/jonas_soft_robot_data/trunk_adiabatic/origin"
rest_file = join(robot_dir, 'rest_qv.pkl')

Compute observables (delay embedding)

In [ ]:
N_DELAY = 4
assemble_observables = lambda oData: utils.delayEmbedding(oData, up_to_delay=N_DELAY)

Predict open-loop trajectories using the interpolated adiabatic SSM model

In [ ]:
test_results = {}
test_trajectory_folders = ['open-loop_circle']
test_trajectories = []
for test_traj in test_trajectory_folders:
    traj_dir = join(robot_dir, "dataCollection", test_traj)
    (t, z), u = utils.import_pos_data(data_dir=traj_dir,
                                      rest_file=rest_file,
                                      output_node=TIP_NODE, return_inputs=True, traj_index=0)
    # z = (z.T - q_eq).T
    # u = (u.T - u_eq).T
    y = assemble_observables(z)
    print("y.shape:", y.shape)
    print("u.shape:", u.shape)

    test_trajectories.append({
            'name': test_traj,
            't': t,
            'z': z,
            'u': u,
            'y': y
        })

In [ ]:
# def interpolate_coeffs(xy, coeffs, method="linear"):
#     xy = np.stack(xy)
#     coeffs = np.stack(coeffs)
#     return interpolator(xy[0], xy[1])

# coeffs = [np.random.random((7, 12)) for i in range(5)]
# print(interpolate_coeffs(np.zeros((2, 4)), coeffs).shape)

In [ ]:
def interpolate_coeffs(xy, coeff_name):
    if coeff_name not in interpolation.keys():
        raise RuntimeError("No interpolation available for these coefficents: {coeff_name}")
    # return interpolation[coeff_name](xy).squeeze()
    if coeff_name == "W":
        return w_coeff[0]
    elif coeff_name == "R":
        return r_coeff[0]
    elif coeff_name == "V":
        return Vde[0]
    elif coeff_name == "B":
        return B[0]
    elif coeff_name == "u_bar":
        return np.zeros(8)
    elif coeff_name == "q_bar":
        return np.zeros(3)
    else:
        raise RuntimeError("No interpolation available for these coefficents: {coeff_name}")

In [ ]:
def advect_adiabaticRD_with_inputs(t, y0, u, y, know_target=True):
    # dt = 0.001
    # uInterpFun = interp1d(t, u, axis=1, fill_value="extrapolate")
    # yInterpFun = interp1d(t, y, axis=1, fill_value="extrapolate")
    # N = int(t[-1]/dt) # len(t)-1
    # t = np.linspace(t[0], t[-1], N+1)
    # u = uInterpFun(t)
    # y = yInterpFun(t)
    # print("u.shape:", u.shape)
    dt = 0.01
    N = len(t)-1
    x = np.full((6, N+1), np.nan)
    y_pred = np.full((15, N+1), np.nan)
    y_pred[:, 0] = y0
    y_bar = np.full((15, N+1), np.nan)
    u_bar = np.full((8, N+1), np.nan)
    xdot = np.full((6, N+1), np.nan)
    weights = np.full((N_models, N+1), np.nan)

    for i in range(N):
        # compute the weights used at this timestep
        try:
            simplex = tri.find_simplex(y[:2, i])
            b = tri.transform[simplex,:2] @ (y[:2, i] - tri.transform[simplex,2])
            c = np.r_[b, 1 - b.sum()]
            point_idx = tri.simplices[simplex]
            weights[point_idx, i] = c
            # assert np.all(c > 0)
            # compute and integrate the dynamics at this timestep
            y_bar[:, i] = np.tile(interpolate_coeffs(y[:2, i], 'q_bar'), 1 + N_DELAY)
            u_bar[:, i] = interpolate_coeffs(y[:2, i], 'u_bar')
            # x[i] = V[i]^T @ (y[i] - y_bar[i])
            # if i == 0:
            x[:, i] = interpolate_coeffs(y[:2, i], 'V').T @ (y_pred[:, i].T - y_bar[:, i]) # y[:, i]) # 
            # xdot[i] = R(x[i]) + B[i] @ (u[i] - u_bar[i])
            xdot[:, i] = (interpolate_coeffs(y[:2, i], 'R') @ utils.phi(x[:, i].reshape(-1, 1), ROMOrder[0])).flatten() + interpolate_coeffs(y[:2, i], 'B') @ (u[:, i] - u_bar[:, i]) # -0.5 * np.eye(6) @ x[:, i] # -0.001 * np.ones(6) # 
            # forward Euler: x[i+1] = x[i] + dt * xdot[i]
            x[:, i+1] = x[:, i] + dt * xdot[:, i]
            # y[i+1] = W(x[i+1]) + y_bar[i]
            y_pred[:, i+1] = (interpolate_coeffs(y[:2, i], 'W') @ utils.phi(x[:, i+1].reshape(-1, 1), SSMOrder[0])).T + y_bar[:, i]
            if not know_target:
                y[:, i+1] = y_pred[:, i+1]
        except Exception as e:
            print("i =", i)
            print("y_pred[i-1]:", y_pred[:, i-1])
            print("y[i-1]:", y[:, i-1])
            print("x[i-1]:", x[:, i-1])
            print("=======================================")
            print("y_pred[i]:", y_pred[:, i])
            print("y[i]:", y[:, i])
            print("x[i]:", x[:, i])
            print("=======================================")
            print("y_pred[i+1]:", y_pred[:, i+1])
            print("y[i+1]:", y[:, i+1])
            print("x[i+1]:", x[:, i+1])
            print("=======================================")
            print(e)
            break
            # raise e
    return t, x, y_pred, xdot, y_bar, u_bar, weights

In [ ]:
from scipy.interpolate import interp1d

def advect_adiabaticRD_with_inputs(t, y0, u, y, know_target=True):
    uInterpFun = interp1d(t, u, axis=1, fill_value="extrapolate")
    def R(t, x):
        y = (interpolate_coeffs(y[:2, i], 'W') @ utils.phi(x[:, i+1].reshape(-1, 1), SSMOrder[0])).T + y_bar[:, i]
        simplex = tri.find_simplex(y[:2])
        b = tri.transform[simplex,:2] @ (y[:2] - tri.transform[simplex,2])
        c = np.r_[b, 1 - b.sum()]
        point_idx = tri.simplices[simplex]
        weights[point_idx, i] = c
        # compute and integrate the dynamics at this timestep
        y_bar = np.tile(interpolate_coeffs(y[:2], 'q_bar'), 1 + N_DELAY)
        u_bar = interpolate_coeffs(y[:2], 'u_bar')
        # x[i] = V[i]^T @ (y[i] - y_bar[i])
        x = interpolate_coeffs(y[:2], 'V').T @ (y[:].T - y_bar)
        xdot = (interpolate_coeffs(y[:2], 'R') @ utils.phi(x.reshape(-1, 1), ROMOrder[0])).flatten() + interpolate_coeffs(y[:2], 'B') @ (uInterpFun(t) - u_bar)
        return xdot

In [ ]:
traj = test_trajectories[0]
print(traj['name'])
t, x, y_pred, xdot, y_bar, u_bar, weights = advect_adiabaticRD_with_inputs(traj['t'], traj['y'][:, 0], traj['u'], traj['y'], know_target=True)

In [ ]:
plt.close('all')
# Trajectory xyz vs. t (predicted vs. actual)
axs = plot.traj_xyz_txyz(t,
                            y_pred[0, :], y_pred[1, :], y_pred[2, :],
                            show=False)
axs = plot.traj_xyz_txyz(traj['t'],
                        traj['z'][0, :], traj['z'][1, :], traj['z'][2, :],
                        color="tab:orange", axs=axs, show=False)
axs[-1].legend(["Predicted trajectory", "Actual trajectory"])
for ax in axs:
    ax.set_ylim(-35, 35)
plt.suptitle(r"Trajectory (predicted vs. actual)")
plt.tight_layout()
plt.show()

# Trajectory 2D (predicted vs. actual)
ax = plot.traj_2D_xy(y_pred[0, :], y_pred[1, :],
                      show=False)
ax = plot.traj_2D_xy(traj['z'][0, :], traj['z'][1, :],
                        color="tab:orange", ax=ax, show=False)
ax.set_xlim(-35, 35)
ax.set_ylim(-35, 35)
ax.legend(["Predicted trajectory", "Actual trajectory"])
plt.suptitle(r"Trajectory (predicted vs. actual)")
plt.show()

# weights
axs = plot.adiabatic_model_weights(t, weights, model_names)

# psi_bar
axs = plot.traj_xyz_txyz(t,
                        y_bar[0, :], y_bar[1, :], y_bar[2, :],
                        xyz_names=[r"$\bar{\psi}_1(t)$", r"$\bar{\psi}_2(t)$", r"$\bar{\psi}_3(t)$"],
                        show=False, rotate_yticks=True)
plt.suptitle(r"$\bar{\psi}(t)$")
plt.show()

# xi_dot
axs = plot.traj_xyz_txyz(t,
                        xdot[0, :], xdot[1, :], xdot[2, :],
                        xyz_names=[r"$\dot{\xi}_1(t)$", r"$\dot{\xi}_2(t)$", r"$\dot{\xi}_3(t)$"],
                        show=False, rotate_yticks=True)
# for ax in axs:
#     ax.set_ylim(-100, 100)
plt.suptitle(r"$\dot{\xi}(t)$")
plt.show()

# xi
axs = plot.traj_xyz_txyz(t,
                        x[0, :], x[1, :], x[2, :],
                        xyz_names=[r"$\xi_1(t)$", r"$\xi_2(t)$", r"$\xi_3(t)$"],
                        show=False, rotate_yticks=True)
# for ax in axs:
#     ax.set_ylim(-100, 100)
plt.suptitle(r"$\xi(t)$")
plt.show()
